# Predicting HDB resale prices with sci-kit learn

<ins>Date Created/ Submitted: 30 Dec 18</ins>

### Part 1: Getting the Data
The code is split into 3 parts, namely getting the data, analysing the data and predicting prices.

This notebook covers loading the data, and the steps included are:
* Setting up the python environment
* Loading the datasets
* Merging the datasets
* Export final dataset as csv

Sit back relax drink a cup of coffee, click run all to get the data in local for loading, exploring and modelling.

### 1. Setting Up the Environment

Due to the wide range of functionality explored for this assignment, many additional packages were installed. It is necessary to do a pip install or conda install if you encounter any error importing the libraries.

In [1]:
# Import libraries required
import os, requests, zipfile, io
from six.moves import urllib
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
import geopy
from geopy import distance

### 2. Loading the Data

HDB resale transaction table as well as three supplementary tables were used for this assignment. Data retrieval and saving to local, as well as feature generation and merging of datasets are performed in Python below.

The final output in this sub-process is the merged dataset, which is written to csv. The csv file will then be used for subsequent exploration. This is to ensure for later exploration and modeling the same data is used, as well as to save runtime. In particular, determining the nearest amenity for each HDB block and its distance is computationally expensive (in terms of time). Otherwise, newly retrieved data might have been updated and/or links no longer working.

Consequently, affected code in this sub-process will be commented out.  Feel free to retrieve updated datasets if you so wish.

### a. Retrieving HDB Resale Transactions

For the assignment, we will be looking at the hdb resale transactions from Jan 2015 obtained from data.gov.sg. As of writing, the dataset consist of 77674 rows and 11 columns (month, town, flat_type, block	street_name, storey_range, floor_area_sqm, flat_model	lease_commence_date, remaining_lease, resale_price) which we will further look into during data exploration.

In [2]:
HDBRESALE_URL = "https://data.gov.sg/dataset/7a339d20-3c57-4b11-a695-9348adfd7614/download"
HDBRESALE_PATH = os.path.join("datasets","hdb")
HDBRS_MEMBER = "resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv"

In [3]:
#Download/ Update and unzip hdb resale data to the designated path
def fetch_hdbresale_data(hdbresale_url=HDBRESALE_URL, hdbresale_path=HDBRESALE_PATH, hdbrs_member=HDBRS_MEMBER):
    if not os.path.isdir(hdbresale_path):
        os.makedirs(hdbresale_path)
    retrieve_hdbresale = requests.get(hdbresale_url, allow_redirects=True, stream=True)
    with zipfile.ZipFile(io.BytesIO(retrieve_hdbresale.content)) as zip:
        zip.extract(hdbrs_member, path=hdbresale_path)

In [4]:
#If the data has already been downloaded, comment out this cell or skip running this cell
fetch_hdbresale_data()

In [5]:
#Load hdb resale data from local - we are only looking at data from Jan 2015
def load_hdbresale(hdbresale_path=HDBRESALE_PATH,hdbrs_member=HDBRS_MEMBER):
    csv_path = os.path.join(hdbresale_path, hdbrs_member)
    return pd.read_csv(csv_path)

In [6]:
hdbresale = load_hdbresale()

In [7]:
#View the data
display(hdbresale.shape)
display(hdbresale.head(5))

(77674, 11)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0


### b. Obtaining the Coordinates for HDB

In order to compute proximity/ distance of HDB to train stations, we need to have geospatial coordinates of each HDB block. We obtained a 
<a href="https://github.com/xkjyeah/singapore-postal-codes">data dump</a> of Singapore Postal Codes for Buildings, which contains coordinate information. 

In [8]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/xkjyeah/singapore-postal-codes/master/"
POSTALCODE_URL = DOWNLOAD_ROOT + "buildings.json"
POSTALCODE_PATH = os.path.join("datasets","postalcode")

In [9]:
#Download postal code data dump
def fetch_postalcode_data(postalcode_url=POSTALCODE_URL, postalcode_path=POSTALCODE_PATH):
    if not os.path.isdir(postalcode_path):
        os.makedirs(postalcode_path)
    json_path = os.path.join(postalcode_path, "buildings.json")
    urllib.request.urlretrieve(postalcode_url, json_path)

In [10]:
#If the data has already been downloaded, comment out this cell or skip running this cell
fetch_postalcode_data()

In [11]:
#Load postal code data
def load_postalcodes(postalcode_path=POSTALCODE_PATH):
    json_path = os.path.join(postalcode_path, "buildings.json")
    return pd.read_json(json_path)

In [12]:
postalcodes = load_postalcodes()

In [13]:
display(postalcodes.shape)
display(postalcodes.head(5))

(141782, 11)

,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y
0,1 STRAITS BOULEVARD SINGAPORE CHINESE CULTURAL...,1,SINGAPORE CHINESE CULTURAL CENTRE,1.275829,103.849576,103.849576,18906,STRAITS BOULEVARD,SINGAPORE CHINESE CULTURAL CENTRE,29809.365407,28700.236127
1,11A STRAITS BOULEVARD TEMPORARY SITE OFFICE SI...,11A,TEMPORARY SITE OFFICE,1.274950,103.851665,103.851665,18907,STRAITS BOULEVARD,TEMPORARY SITE OFFICE,30041.838898,28602.987244
2,5A MARINA GARDENS DRIVE SINGAPORE 018910,5A,NIL,1.279587,103.868956,103.868956,18910,MARINA GARDENS DRIVE,5A MARINA GARDENS DRIVE SINGAPORE 018910,31966.120787,29115.753373
3,100A CENTRAL BOULEVARD TEMPORARY SITE OFFICE S...,100A,TEMPORARY SITE OFFICE,1.273682,103.860075,103.860075,18915,CENTRAL BOULEVARD,TEMPORARY SITE OFFICE,30977.832467,28462.795296
4,2 CENTRAL BOULEVARD CENTRAL BOULEVARD TOWERS S...,2,CENTRAL BOULEVARD TOWERS,1.279785,103.851529,103.851529,18916,CENTRAL BOULEVARD,CENTRAL BOULEVARD TOWERS,30026.620277,29137.649384


In order to merge the datasets, we need to have common key fields. Although both datasets have a blk and street/ road name, the text format in the postal code data dump is inconsistent to that in the HDB datasets (e.g. 'AVE' 'AVENUE, 'ST' 'STREET'). Thus, we normalize the text before merging the datasets. Only the coordinate information is retained. 

In [14]:
#Define function to normalize street names
def normalizeStreet(inputValue,case='u'):
    '''
    if case=='l', returns lowercase
    if case=='u', returns uppercase
    else returns proper case
    '''
    case = case[0].upper()
    abbv = suffixDict()
    words = inputValue.split()
    for i,word in enumerate(words):
#        if not i==0:
        w = word.upper()
        rep = abbv[w] if w in abbv.keys() else words[i]
        words[i] = rep.upper() if case == 'u' else rep.lower() if case == 'l' else (rep[0].upper() + rep[1:])
    return ' '.join(words)

def suffixDict():
    return {'AVENUE':'AVE','BOULEVARD':'BLVD','STREET':'ST','ROAD':'RD','NORTH':'NTH','SOUTH':'STH','CENTRAL':'CTRL','DRIVE':'DR','BUKIT':'BT','JALAN':'JLN','CRESCENT':'CRES','HEIGHTS':'HTS','CLOSE':'CL','PLACE':'PL','UPPER':'UPP','LORONG':'LOR','TANJONG':'TG','GARDEN':'GDN','GARDENS':'GDNS','PARK':'PK','SAINT':'ST.','TERRACE':'TER','COMMONWEALTH':"C'WEALTH",'KAMPONG':'KG','MARKET':'MKT'}

In [15]:
#Normalize road names in postal code data
postalcodes['ROAD_NAMES'] = list(map(normalizeStreet,postalcodes.loc[:,'ROAD_NAME'].tolist()))

In [16]:
#View 10 random records with original road name and normalized road name
postalcodes.loc[np.random.randint(1,(postalcodes.shape[0]+1),size=10),['ROAD_NAME','ROAD_NAMES']]

,ROAD_NAME,ROAD_NAMES
44529,UPPER BOON KENG ROAD,UPP BOON KENG RD
50086,LORONG SALLEH,LOR SALLEH
15800,JELLICOE ROAD,JELLICOE RD
125148,WOODLANDS CRESCENT,WOODLANDS CRES
80356,TAMPINES STREET 92,TAMPINES ST 92
67357,NEW UPPER CHANGI ROAD,NEW UPP CHANGI RD
75886,LOYANG RISE,LOYANG RISE
12705,ENG HOON STREET,ENG HOON ST
72370,CHANGI SOUTH AVENUE 2,CHANGI STH AVE 2
101958,JALAN TERUBOK,JLN TERUBOK


We create a new dataframe containing a subset of the postal code data - only those blk and road names contained in the hdb resale dataset is retained. The other irrelevant data rows will not be required.

We also use drop duplicates to avoid introducing/ keeping any duplicates. 

In [17]:
hdb_coordinates = postalcodes.loc[:,['BLK_NO','ROAD_NAMES','X','Y']].merge(hdbresale[['block','street_name']],how='inner',right_on=['block','street_name'],left_on=['BLK_NO','ROAD_NAMES']).drop_duplicates(subset=['block','street_name'], keep='last').drop(['block','street_name'],axis=1)

In [18]:
hdb_coordinates.rename(columns={'BLK_NO':'block','ROAD_NAMES':'street_name'}, inplace=True)

In [19]:
display(hdb_coordinates.shape)
display(hdb_coordinates.head(5))

(8500, 4)

,block,street_name,X,Y
19,4,SAGO LANE,29067.241846,29360.522264
47,32,NEW MKT RD,29030.582415,29698.658411
81,34,UPP CROSS ST,29300.205134,29610.969428
90,532,UPP CROSS ST,29409.763097,29625.120013
101,533,UPP CROSS ST,29342.322791,29666.277449


We transform the pandas dataframe into geopandas dataframe to facilitate performing of geospatial plotting, transformation and analysis.

In [20]:
hdb_coordinates['geometry_hdb'] = list(zip(hdb_coordinates.X, hdb_coordinates.Y))
hdb_coordinates['geometry_hdb'] = hdb_coordinates['geometry_hdb'].apply(Point)
hdb_coordinates = gpd.GeoDataFrame(hdb_coordinates, geometry='geometry_hdb')

We take a look at the hdb coordinates geopandas dataframe. Essentially it is exactly the same as a pandas dataframe but with an additional geometry attribute.

In [21]:
display(hdb_coordinates.shape)
display(hdb_coordinates.head(5))

(8500, 5)

,block,street_name,X,Y,geometry_hdb
19,4,SAGO LANE,29067.241846,29360.522264,POINT (29067.2418462165 29360.5222637262)
47,32,NEW MKT RD,29030.582415,29698.658411,POINT (29030.5824145211 29698.65841051)
81,34,UPP CROSS ST,29300.205134,29610.969428,POINT (29300.205133812 29610.9694283661)
90,532,UPP CROSS ST,29409.763097,29625.120013,POINT (29409.7630974087 29625.1200134344)
101,533,UPP CROSS ST,29342.322791,29666.277449,POINT (29342.3227910118 29666.2774491161)


Looking at the geometry, note that the X and Y coordinates are larger numbers than usual. This is due to Singapore using the coordinate reference system SVY21 (epsg: 3414). We transform these to the coordinate reference system WGS84 (epsg: 4326) and extract the longitude and latitude points.

In [22]:
#define the crs in coordinates geodataframe
hdb_coordinates.crs = {'init':'epsg:3414'}
hdb_coordinates = hdb_coordinates.to_crs({'init':'epsg:4326'})

In [23]:
hdb_coordinates['latitude']=hdb_coordinates['geometry_hdb'].y
hdb_coordinates['longitude']=hdb_coordinates['geometry_hdb'].x

### c. Derive Proximity to Nearest Train Station

We obtain coordinate information of the train stations. And finally, we are able to compute the distance to the nearest train station for each hdb block. First we retrieve and unzip the data. Then we load the dataset as a geopandas dataframe.

In [24]:
DOWNLOAD_ROOT = "https://www.mytransport.sg/content/dam/datamall/datasets/Geospatial/"
TRAIN_URL = DOWNLOAD_ROOT + "TrainStation.zip"
TRAIN_PATH = os.path.join("datasets","train")

In [25]:
#Download train station data
def fetch_train_data(train_url=TRAIN_URL, train_path=TRAIN_PATH):
    if not os.path.isdir(train_path):
        os.makedirs(train_path)
    retrieve_train = requests.get(train_url)
    with zipfile.ZipFile(io.BytesIO(retrieve_train.content)) as zip:
        for zip_info in zip.infolist():
            if zip_info.filename[-1] == '/':
                continue
            zip_info.filename = os.path.basename(zip_info.filename)
            zip.extract(zip_info, path=train_path)

In [26]:
#If the data has already been downloaded, comment out this cell or skip running this cell
fetch_train_data()

In [27]:
#Read and load train data, specifically, the shp file
def load_train(train_path=TRAIN_PATH):
    for root,dirs,files in os.walk(train_path):
        if files:
            for file in files:
                if os.path.splitext(file)[1] == '.shp':
                    filename = file
                
    shp_path = os.path.join(train_path, filename)
    return gpd.read_file(shp_path)

In [28]:
trainstns = load_train()

In [29]:
display(trainstns.shape)
display(trainstns.head(5))

(183, 4)

,OBJECTID,STN_NAME,STN_NO,geometry
0,1,EUNOS MRT STATION,EW7,POINT (35782.95529999956 33560.07760000043)
1,2,CHINESE GARDEN MRT STATION,EW25,POINT (16790.74660000019 36056.30189999938)
2,3,KHATIB MRT STATION,NS14,POINT (27962.31080000009 44352.56799999997)
3,4,KRANJI MRT STATION,NS7,POINT (20081.69739999995 45214.54790000059)
4,5,REDHILL MRT STATION,EW18,POINT (26163.47800000012 30218.81959999911)


Similarly, we convert the coordinate reference system for the train stations dataset

In [30]:
trainstns.crs = {'init':'epsg:3414'}
trainstns = trainstns.to_crs({'init':'epsg:4326'})

We only want to compute distances for mrt stations. This is based off the assumption that the distances from an mrt station is more valuable then an lrt station, given that an lrt station is similar to a bus stop that brings you to the town center.

In [31]:
mrtstn = trainstns[trainstns['STN_NO'].str.contains('NS|EW|NE|CC|DT')]

We define a function nearest which returns us with the id column for the nearest mrt station which we store in the coordinates dataset. This code will take a while to run.

In [32]:
def nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
    """Find the nearest point and return the corresponding value from specified column."""
    # Find the geometry that is closest
    nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
    # Get the corresponding value from df2 (matching is based on the geometry)
    value1 = df2[nearest][src_column].get_values()[0]
    return value1

In [33]:
mrt_unary_union = mrtstn.unary_union

In [34]:
hdb_coordinates['nearstn_id']= hdb_coordinates.apply(nearest, geom_union=mrt_unary_union, df1=hdb_coordinates, df2=mrtstn, geom1_col='geometry_hdb',geom2_col='geometry', src_column='OBJECTID', axis=1)

In [35]:
display(hdb_coordinates.shape)
display(hdb_coordinates.head(5))

(8500, 8)

,block,street_name,X,Y,geometry_hdb,latitude,longitude,nearstn_id
19,4,SAGO LANE,29067.241846,29360.522264,POINT (103.8429081594952 1.281800622422588),1.281801,103.842908,131
47,32,NEW MKT RD,29030.582415,29698.658411,POINT (103.8425787713264 1.284858608300219),1.284859,103.842579,131
81,34,UPP CROSS ST,29300.205134,29610.969428,POINT (103.8450014348408 1.284065569950329),1.284066,103.845001,132
90,532,UPP CROSS ST,29409.763097,29625.120013,POINT (103.8459858569331 1.284193538161627),1.284194,103.845986,132
101,533,UPP CROSS ST,29342.322791,29666.277449,POINT (103.8453798809252 1.284565754507525),1.284566,103.845380,132


Now we obtained the nearest mrt station id for each hdb block, street pair, we merge the mrt station coordinates.

In [36]:
hdb_coordinates = hdb_coordinates.merge(trainstns,how='left',left_on='nearstn_id',right_on='OBJECTID',suffixes=('_hdb','_mrt'))

With both the hdb and train coordinates in the same dataframe, we can easily compute the distance between the points. And after obtaining the distance measure, irrelevant columns are dropped

In [37]:
hdb_coordinates['nearstn_dist'] = hdb_coordinates.apply(
    (lambda row: distance.distance(
        (row['geometry_hdb'].y, row['geometry_hdb'].x),
        (row['geometry'].y, row['geometry'].x)
    ).kilometers),
    axis=1
)

In [38]:
hdb_coordinates = hdb_coordinates.drop(['OBJECTID','STN_NAME','STN_NO','nearstn_id','geometry'],axis=1)

In [39]:
display(hdb_coordinates.shape)
display(hdb_coordinates.head(5))

(8500, 8)

,block,street_name,X,Y,geometry_hdb,latitude,longitude,nearstn_dist
0,4,SAGO LANE,29067.241846,29360.522264,POINT (103.8429081594952 1.281800622422588),1.281801,103.842908,0.288783
1,32,NEW MKT RD,29030.582415,29698.658411,POINT (103.8425787713264 1.284858608300219),1.284859,103.842579,0.109324
2,34,UPP CROSS ST,29300.205134,29610.969428,POINT (103.8450014348408 1.284065569950329),1.284066,103.845001,0.097264
3,532,UPP CROSS ST,29409.763097,29625.120013,POINT (103.8459858569331 1.284193538161627),1.284194,103.845986,0.181501
4,533,UPP CROSS ST,29342.322791,29666.277449,POINT (103.8453798809252 1.284565754507525),1.284566,103.845380,0.105321


### d. Derive Proximity to Nearest Marker/ Hawker

We obtain address information on market and hawker centers. We determine its coordinates, and similarly, will beable to compute the distance to the nearest market and hawker center for each hdb block.

In [40]:
MKTHWK_URL = "https://data.gov.sg/dataset/b6083025-58a6-41a4-8066-c51a3282218f/download"
MKTHWK_PATH = os.path.join("datasets","mkthwk")
MKTHWK_MEMBER = 'list-of-government-markets-hawker-centres.csv'

In [41]:
#Download mkt hwk info
def fetch_mkthwk_data(mkthwk_url=MKTHWK_URL, mkthwk_path=MKTHWK_PATH, mkthwk_member=MKTHWK_MEMBER):
    if not os.path.isdir(mkthwk_path):
        os.makedirs(mkthwk_path)
    retrieve_mkthwkinfo = requests.get(mkthwk_url, allow_redirects=True, stream=True)
    with zipfile.ZipFile(io.BytesIO(retrieve_mkthwkinfo.content)) as zip:
        zip.extract(mkthwk_member, path=mkthwk_path)

In [42]:
#If the data has already been downloaded, comment out this cell or skip running this cell
fetch_mkthwk_data()

In [43]:
def load_mkthwkinfo(mkthwk_path=MKTHWK_PATH, mkthwk_member=MKTHWK_MEMBER):
    csv_path = os.path.join(mkthwk_path,mkthwk_member)
    return pd.read_csv(csv_path)

In [44]:
mkthwk = load_mkthwkinfo()

In [45]:
display(mkthwk.shape)
display(mkthwk.head(5))

(107, 7)

,name_of_centre,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls
0,Adam Road Food Centre,"2, Adam Road, S(289876)",HC,Government,32,32,0
1,Amoy Street Food Centre,"National Development Building, Annex B, Telok ...",HC,Government,135,134,1
2,Bedok Food Centre,"1, Bedok Road, S(469572)",HC,Government,32,32,0
3,Beo Crescent Market,"38A, Beo Crescent, S(169982)",MHC,Government,94,32,62
4,Berseh Food Centre,"166, Jalan Besar, S(208877)",HC,Government,66,66,0


We need to extract the postal code information in order to look up the postal code table and obtain the market and hawker center coordinates. After obtaining the coordinates, we drop all irrelevant columns.

In [46]:
mkthwk['postal_code'] = mkthwk['location_of_centre'].str.extract(r"S\(([0-9]+)[\)/]").iloc[:,0]

In [47]:
postalcodes['POSTAL']=postalcodes['POSTAL'].astype(str).str.zfill(6)

In [48]:
mkthwk = mkthwk.merge(postalcodes[['POSTAL','LATITUDE','LONGITUDE']].drop_duplicates('POSTAL'),how='left',right_on=str('POSTAL').zfill(6),left_on='postal_code').drop(['POSTAL','location_of_centre','type_of_centre','owner','no_of_stalls','no_of_cooked_food_stalls','no_of_mkt_produce_stalls','postal_code'],axis=1).drop_duplicates('name_of_centre')

We turn the market and hawker pandas dataframe into geopandas dataframe

In [49]:
mkthwk['geometry'] = list(zip(mkthwk.LONGITUDE, mkthwk.LATITUDE))
mkthwk['geometry'] = mkthwk['geometry'].apply(Point)
mkthwk = gpd.GeoDataFrame(mkthwk, geometry='geometry').drop(['LONGITUDE','LATITUDE'],axis=1)

In [50]:
mkthwk.crs = {'init':'epsg:4326'}

In [51]:
display(mkthwk.shape)
display(mkthwk.head(5))

(107, 2)

,name_of_centre,geometry
0,Adam Road Food Centre,POINT (103.814165924136 1.3241598525956)
1,Amoy Street Food Centre,POINT (103.846652482254 1.27933986365712)
2,Bedok Food Centre,POINT (103.955480577498 1.32034716669857)
3,Beo Crescent Market,POINT (103.827353973547 1.28883084168339)
4,Berseh Food Centre,POINT (103.856888783769 1.30734410946374)


In [52]:
mkthwk_unary_union = mkthwk.unary_union

In [53]:
hdb_coordinates['nearmkthwk_name']= hdb_coordinates.apply(nearest, geom_union=mkthwk_unary_union, df1=hdb_coordinates, df2=mkthwk, geom1_col='geometry_hdb',geom2_col='geometry', src_column='name_of_centre', axis=1)

In [54]:
hdb_coordinates = hdb_coordinates.merge(mkthwk,how='left',left_on='nearmkthwk_name',right_on='name_of_centre',suffixes=('_hdb','_mkthwk'))

In [55]:
display(hdb_coordinates.shape)
display(hdb_coordinates.head(5))

(8500, 11)

,block,street_name,X,Y,geometry_hdb,latitude,longitude,nearstn_dist,nearmkthwk_name,name_of_centre,geometry
0,4,SAGO LANE,29067.241846,29360.522264,POINT (103.8429081594952 1.281800622422588),1.281801,103.842908,0.288783,Chinatown Market,Chinatown Market,POINT (103.843193109493 1.28227638466274)
1,32,NEW MKT RD,29030.582415,29698.658411,POINT (103.8425787713264 1.284858608300219),1.284859,103.842579,0.109324,People's Park Food Centre,People's Park Food Centre,POINT (103.842599920236 1.28485588430581)
2,34,UPP CROSS ST,29300.205134,29610.969428,POINT (103.8450014348408 1.284065569950329),1.284066,103.845001,0.097264,Hong Lim Market & Food Centre,Hong Lim Market & Food Centre,POINT (103.845823802043 1.28529907736774)
3,532,UPP CROSS ST,29409.763097,29625.120013,POINT (103.8459858569331 1.284193538161627),1.284194,103.845986,0.181501,Hong Lim Market & Food Centre,Hong Lim Market & Food Centre,POINT (103.845823802043 1.28529907736774)
4,533,UPP CROSS ST,29342.322791,29666.277449,POINT (103.8453798809252 1.284565754507525),1.284566,103.845380,0.105321,Hong Lim Market & Food Centre,Hong Lim Market & Food Centre,POINT (103.845823802043 1.28529907736774)


We similarly compute the distance and drop the irrelevant columns

In [56]:
hdb_coordinates['nearmkthwk_dist'] = hdb_coordinates.apply(
    (lambda row: distance.distance(
        (row['geometry_hdb'].y, row['geometry_hdb'].x),
        (row['geometry'].y, row['geometry'].x)
    ).kilometers),
    axis=1
)

In [57]:
hdb_coordinates=hdb_coordinates.drop(['nearmkthwk_name','name_of_centre','geometry'], axis=1)

In [58]:
display(hdb_coordinates.shape)
display(hdb_coordinates.head(5))

(8500, 9)

,block,street_name,X,Y,geometry_hdb,latitude,longitude,nearstn_dist,nearmkthwk_dist
0,4,SAGO LANE,29067.241846,29360.522264,POINT (103.8429081594952 1.281800622422588),1.281801,103.842908,0.288783,0.061427
1,32,NEW MKT RD,29030.582415,29698.658411,POINT (103.8425787713264 1.284858608300219),1.284859,103.842579,0.109324,0.002373
2,34,UPP CROSS ST,29300.205134,29610.969428,POINT (103.8450014348408 1.284065569950329),1.284066,103.845001,0.097264,0.164256
3,532,UPP CROSS ST,29409.763097,29625.120013,POINT (103.8459858569331 1.284193538161627),1.284194,103.845986,0.181501,0.123568
4,533,UPP CROSS ST,29342.322791,29666.277449,POINT (103.8453798809252 1.284565754507525),1.284566,103.845380,0.105321,0.094952


### e. Derive Proximity to Nearest Primary School

We obtain address information on schools. In view of the proximity benefits when registering a child in primary school, we filter only primary school. We determine its coordinates, and similarly, will be able to compute the distance to the nearest primary school for each hdb block.

In [59]:
SCH_URL = "https://data.gov.sg/dataset/c004b703-5e64-47db-a504-e60e74fd3b32/download"
SCH_PATH = os.path.join("datasets","sch")
SCH_MEMBER = 'general-information-of-schools.csv'

In [60]:
#Download sch info
def fetch_schinfo_data(sch_url=SCH_URL, sch_path=SCH_PATH, sch_member=SCH_MEMBER):
    if not os.path.isdir(sch_path):
        os.makedirs(sch_path)
    retrieve_schinfo = requests.get(sch_url, allow_redirects=True, stream=True)
    with zipfile.ZipFile(io.BytesIO(retrieve_schinfo.content)) as zip:
        zip.extract(sch_member, path=sch_path)

In [61]:
#If the data has already been downloaded, comment out this cell or skip running this cell
fetch_schinfo_data()

In [62]:
def load_schinfo(sch_path=SCH_PATH, sch_member=SCH_MEMBER):
    csv_path = os.path.join(sch_path,sch_member)
    return pd.read_csv(csv_path)

In [63]:
sch = load_schinfo()

In [64]:
display(sch.shape)
display(sch.head(5))

(358, 35)

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,special_sdp_offered
0,NATIONAL JUNIOR COLLEGE,www.nationaljc.moe.edu.sg,37 HILLCREST ROAD,288913,64661144,na,64684535,na,NJC@MOE.EDU.SG,"BOTANIC GARDENS MRT, TAN KAH KEE MRT, SIXTH AV...",...,FULL DAY,MIXED LEVEL,No,No,No,Yes,Chinese,Malay,Tamil,"For the Integrated Programme (Junior High), we..."
1,TEMASEK JUNIOR COLLEGE,http://temasekjc.moe.edu.sg,22 BEDOK SOUTH ROAD,469278,64428066,na,64428762,na,TEMASEK_JC@MOE.EDU.SG,BEDOK MRT,...,FULL DAY,MIXED LEVEL,No,No,No,Yes,Chinese,Malay,Tamil,TJC 6-Year Integrated Programme provides a sea...
2,JURONG JUNIOR COLLEGE,http://www.jurongjc.moe.edu.sg,800 CORPORATION ROAD,649809,65624611,na,65624505,na,jjc@moe.edu.sg,"Lakeside, Boon Lay",...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,Chinese,Malay,Tamil,1. Experiential Learning Programmes (ELP) 2. E...
3,ANDERSON JUNIOR COLLEGE,ajc.moe.edu.sg.,4500 ANG MO KIO AVENUE 6,569843,64596822,na,64598734,na,anderson_jc@moe.edu.sg,Yio Chu Kang,...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,na,na,na,"Science research, service-learning, student le..."
4,VICTORIA JUNIOR COLLEGE,http://www.victoriajc.moe.edu.sg/,20 MARINE VISTA,449035,64485011,na,64438337,na,victoria_jc@moe.edu.sg,"Nearest MRT Stations: Kembangan, Eunos",...,FULL DAY,JUNIOR COLLEGE,No,No,No,Yes,Chinese,Malay,Tamil,Arts Programme Beyond Borders Programme Highe...


Filter pri school based on mainlevel code and mixed level schools which also admit primary school students.

In [65]:
prisch = sch[sch['mainlevel_code'].str.contains('PRIMARY') | sch['school_name'].str.contains("ST. JOSEPH'S INSTITUTION") | sch['school_name'].str.contains("CATHOLIC HIGH SCHOOL") |  sch['school_name'].str.contains("MARIS STELLA HIGH SCHOOL") |  sch['school_name'].str.contains("CHIJ ST. NICHOLAS GIRLS' SCHOOL")]

In [66]:
display(prisch.shape)
display(prisch.head(5))

(192, 35)

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,special_sdp_offered
18,BUKIT PANJANG PRIMARY SCHOOL,http://www.bukitpanjangpri.moe.edu.sg,109 CASHEW ROAD,679676,67691912,na,67637462,na,BPPS@MOE.EDU.SG,From Pending LRT station to Choa Chu Kang MRT ...,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,"Learning for Life Programme (LLP) -""i2can danc..."
19,HAIG GIRLS' SCHOOL,http://www.haiggirls.moe.edu.sg,51 KOON SENG ROAD,427072,63440293,na,64474169,na,HAIGGIRLSSCH@MOE.EDU.SG,"Eunos, Dakota",...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,LLP - Character and Leadership through Perform...
20,TECK WHYE PRIMARY SCHOOL,http://www.teckwhyepri.moe.edu.sg,11 TECK WHYE WALK,688261,67691025,na,67635191,na,twps@moe.edu.sg,Chua Chu Kang MRT & LRTs,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,Teck Whye Primary's Applied Learning Programme...
21,JURONG PRIMARY SCHOOL,http://www.jurongpri.moe.edu.sg,320 JURONG EAST STREET 32,609476,65618837,na,65641964,na,JPS@MOE.EDU.SG,"Lakeside MRT Station, Jurong East MRT Station",...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,JPS is committed to giving each child a broad ...
22,QUEENSTOWN PRIMARY SCHOOL,http://www.queenstownpri.moe.edu.sg,310 MARGARET DR,149303,64741044,na,64713640,na,qtps@moe.edu.sg,Queenstown MRT,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,"In QtPS, the vision of our Character & Citizen..."


In [67]:
postal_str = prisch.loc[:,'postal_code'].astype(str).str.zfill(6).tolist()
prisch['postal_code'] = postal_str

/Users/alphaniang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [68]:
prisch = prisch.merge(postalcodes[['POSTAL','LATITUDE','LONGITUDE']].drop_duplicates('POSTAL'),how='left',right_on=['POSTAL'],left_on=['postal_code'])
prisch = prisch[['school_name','LATITUDE','LONGITUDE']]

In [69]:
prisch.head(5)

,school_name,LATITUDE,LONGITUDE
0,BUKIT PANJANG PRIMARY SCHOOL,1.373472,103.769317
1,HAIG GIRLS' SCHOOL,1.311943,103.902903
2,TECK WHYE PRIMARY SCHOOL,1.383887,103.753925
3,JURONG PRIMARY SCHOOL,1.348600,103.733157
4,QUEENSTOWN PRIMARY SCHOOL,1.295594,103.807549


In [70]:
prisch['geometry'] = list(zip(prisch.LONGITUDE, prisch.LATITUDE))
prisch['geometry'] = prisch['geometry'].apply(Point)
prisch = gpd.GeoDataFrame(prisch, geometry='geometry').drop(['LONGITUDE','LATITUDE'],axis=1)

In [71]:
prisch.crs = {'init':'epsg:4326'}

In [72]:
prisch_unary_union = prisch.unary_union

In [73]:
hdb_coordinates['nearprisch_name']= hdb_coordinates.apply(nearest, geom_union=prisch_unary_union, df1=hdb_coordinates, df2=prisch, geom1_col='geometry_hdb',geom2_col='geometry', src_column='school_name', axis=1)

In [74]:
hdb_coordinates = hdb_coordinates.merge(prisch,how='left',left_on='nearprisch_name',right_on='school_name',suffixes=('_hdb','_prisch'))

In [75]:
hdb_coordinates['nearprisch_dist'] = hdb_coordinates.apply(
    (lambda row: distance.distance(
        (row['geometry_hdb'].y, row['geometry_hdb'].x),
        (row['geometry'].y, row['geometry'].x)
    ).kilometers),
    axis=1
)

In [76]:
hdb_coordinates = hdb_coordinates.drop(['nearprisch_name','school_name','geometry'],axis=1)

In [77]:
display(hdb_coordinates.shape)
display(hdb_coordinates.head(5))

(8500, 10)

,block,street_name,X,Y,geometry_hdb,latitude,longitude,nearstn_dist,nearmkthwk_dist,nearprisch_dist
0,4,SAGO LANE,29067.241846,29360.522264,POINT (103.8429081594952 1.281800622422588),1.281801,103.842908,0.288783,0.061427,0.772710
1,32,NEW MKT RD,29030.582415,29698.658411,POINT (103.8425787713264 1.284858608300219),1.284859,103.842579,0.109324,0.002373,1.077930
2,34,UPP CROSS ST,29300.205134,29610.969428,POINT (103.8450014348408 1.284065569950329),1.284066,103.845001,0.097264,0.164256,1.103315
3,532,UPP CROSS ST,29409.763097,29625.120013,POINT (103.8459858569331 1.284193538161627),1.284194,103.845986,0.181501,0.123568,1.174424
4,533,UPP CROSS ST,29342.322791,29666.277449,POINT (103.8453798809252 1.284565754507525),1.284566,103.845380,0.105321,0.094952,1.172381


### f. Derive whether Top Schools within 1 km

We identify whether there are any top primary schools within 1km of the hdb block

In [78]:
#https://thesmartlocal.com/read/best-primary-schools
topschlist = ["RAFFLES GIRLS' PRIMARY SCHOOL", "AI TONG SCHOOL", "CATHOLIC HIGH SCHOOL", "CHIJ ST. NICHOLAS GIRLS' SCHOOL", "KONG HWA SCHOOL", "METHODIST GIRLS' SCHOOL (PRIMARY)", "TAO NAN SCHOOL", "HONG WEN SCHOOL", "HENRY PARK PRIMARY SCHOOL", "MARIS STELLA HIGH SCHOOL", "SINGAPORE CHINESE GIRLS' PRIMARY SCHOOL", "ANGLO-CHINESE SCHOOL (PRIMARY)", "ANGLO-CHINESE SCHOOL (JUNIOR)", "HAIG GIRLS' SCHOOL", "FAIRFIELD METHODIST SCHOOL (PRIMARY)", "RIVER VALLEY PRIMARY SCHOOL", "CHIJ (KELLOCK)", "CHONGFU SCHOOL", "CANBERRA PRIMARY SCHOOL", "NORTHLAND PRIMARY SCHOOL", "ROSYTH SCHOOL", "PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY)", "ST. HILDA'S PRIMARY SCHOOL", "PASIR RIS PRIMARY SCHOOL", "TAMPINES PRIMARY SCHOOL", "TEMASEK PRIMARY SCHOOL", "RULANG PRIMARY SCHOOL", "KEMING PRIMARY SCHOOL", "NAN HUA PRIMARY SCHOOL"
]

In [79]:
topprisch = prisch[prisch['school_name'].isin(topschlist)].reset_index().drop('index',axis=1)

In [80]:
topprisch['latitude']=topprisch['geometry'].y
topprisch['longitude']=topprisch['geometry'].x

In [81]:
topprisch.crs = {'init':'epsg:4326'}

In [82]:
topprisch.head(5)

,school_name,geometry,latitude,longitude
0,HAIG GIRLS' SCHOOL,POINT (103.902902557195 1.31194344271385),1.311943,103.902903
1,RAFFLES GIRLS' PRIMARY SCHOOL,POINT (103.806397828938 1.33004178068278),1.330042,103.806398
2,ROSYTH SCHOOL,POINT (103.874693248439 1.37291579398894),1.372916,103.874693
3,HENRY PARK PRIMARY SCHOOL,POINT (103.784296227747 1.31667646178349),1.316676,103.784296
4,TEMASEK PRIMARY SCHOOL,POINT (103.945590673243 1.31770678180901),1.317707,103.945591


In [83]:
from geopy.distance import great_circle
neartopschcount = []
neartopsch = []

for i in range(len(hdb_coordinates)):
    for j in range(len(topprisch)):
        neartopschcount = 0
        prisch_dist=great_circle((hdb_coordinates.loc[i,'latitude'], hdb_coordinates.loc[i,'longitude']),
            (topprisch.loc[j,'latitude'], topprisch.loc[j,'longitude'])).kilometers
        if prisch_dist < 1:
            neartopschcount += 1
    if neartopschcount == 0:
        neartopsch.append('N')
    else:
        neartopsch.append('Y')

In [84]:
hdb_coordinates['neartopsch']=neartopsch

In [85]:
hdb_coordinates.head(5)

,block,street_name,X,Y,geometry_hdb,latitude,longitude,nearstn_dist,nearmkthwk_dist,nearprisch_dist,neartopsch
0,4,SAGO LANE,29067.241846,29360.522264,POINT (103.8429081594952 1.281800622422588),1.281801,103.842908,0.288783,0.061427,0.772710,N
1,32,NEW MKT RD,29030.582415,29698.658411,POINT (103.8425787713264 1.284858608300219),1.284859,103.842579,0.109324,0.002373,1.077930,N
2,34,UPP CROSS ST,29300.205134,29610.969428,POINT (103.8450014348408 1.284065569950329),1.284066,103.845001,0.097264,0.164256,1.103315,N
3,532,UPP CROSS ST,29409.763097,29625.120013,POINT (103.8459858569331 1.284193538161627),1.284194,103.845986,0.181501,0.123568,1.174424,N
4,533,UPP CROSS ST,29342.322791,29666.277449,POINT (103.8453798809252 1.284565754507525),1.284566,103.845380,0.105321,0.094952,1.172381,N


### 3. Merge the Datasets

In [86]:
hdbresale = hdbresale.merge(hdb_coordinates,how='left',left_on=['block','street_name'],right_on=['block','street_name'])

In [87]:
hdbresale.head(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,X,Y,geometry_hdb,latitude,longitude,nearstn_dist,nearmkthwk_dist,nearprisch_dist,neartopsch
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,28478.579445,39676.807613,POINT (103.8376189612301 1.375097468679052),1.375097,103.837619,1.098736,0.175973,0.415219,Y
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,30482.026488,39546.884200,POINT (103.8556213705245 1.373922387034844),1.373922,103.855621,0.806153,0.181499,0.212413,N
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,28539.786998,39505.894691,POINT (103.8381689522579 1.373551792255509),1.373552,103.838169,1.179839,0.120919,0.435846,Y
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,30452.628285,38865.583214,POINT (103.8553571502598 1.367760947203525),1.367761,103.855357,0.628613,0.124075,0.537163,N
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,30717.373319,39292.930731,POINT (103.8577361075271 1.371625702033223),1.371626,103.857736,0.904199,0.386214,0.141448,N


### 4. Write Data to Csv

In the last step, we tidy the dataset and write to csv for later use.

In [88]:
#See the columns in the merged dataset
hdbresale.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'X', 'Y', 'geometry_hdb', 'latitude',
       'longitude', 'nearstn_dist', 'nearmkthwk_dist', 'nearprisch_dist',
       'neartopsch'],
      dtype='object')

In [89]:
#Drop unwanted columns 
hdbresale = hdbresale.drop(['X','Y'], axis=1)

In [90]:
#Final look at the data
hdbresale.head(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,geometry_hdb,latitude,longitude,nearstn_dist,nearmkthwk_dist,nearprisch_dist,neartopsch
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,POINT (103.8376189612301 1.375097468679052),1.375097,103.837619,1.098736,0.175973,0.415219,Y
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,POINT (103.8556213705245 1.373922387034844),1.373922,103.855621,0.806153,0.181499,0.212413,N
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,POINT (103.8381689522579 1.373551792255509),1.373552,103.838169,1.179839,0.120919,0.435846,Y
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,POINT (103.8553571502598 1.367760947203525),1.367761,103.855357,0.628613,0.124075,0.537163,N
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,POINT (103.8577361075271 1.371625702033223),1.371626,103.857736,0.904199,0.386214,0.141448,N


In [91]:
#Write the dataset to csv
hdbresale.to_csv('hdbresale.csv')

### 5. Download Singapore Map Files

In [92]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/yinshanyang/singapore/master/maps/"
SGMAP_URL = DOWNLOAD_ROOT + "0-mainland.geojson"
SGMAP_PATH = os.path.join("datasets","maps")

In [93]:
#Download sg map
def fetch_sgmap(sgmap_url=SGMAP_URL, sgmap_path=SGMAP_PATH):
    if not os.path.isdir(sgmap_path):
        os.makedirs(sgmap_path)
    geojson_path = os.path.join(sgmap_path, "0-mainland.geojson")
    urllib.request.urlretrieve(sgmap_url, geojson_path)

In [94]:
#If the data has already been downloaded, comment out this cell
fetch_sgmap()

In [95]:
SGPA_URL = "https://data.gov.sg/dataset/4d9e3b2f-3f4b-488e-bb3a-3638ca656247/download"
SGPA_PATH = os.path.join("datasets","maps")
SGPA_MEMBER = 'master-plan-2014-planning-area-boundary-web-shp.zip'

In [96]:
#Download sgpamap
def fetch_sgpamap(sgpa_url=SGPA_URL, sgpa_path=SGPA_PATH, sgpa_member=SGPA_MEMBER):
    if not os.path.isdir(sgpa_path):
        os.makedirs(sgpa_path)
    retrieve_sgpamap = requests.get(sgpa_url, allow_redirects=True, stream=True)
    with zipfile.ZipFile(io.BytesIO(retrieve_sgpamap.content)) as zip:
        zip.extract(sgpa_member, path=sgpa_path)

In [97]:
#If the data has already been downloaded, comment out this cell
fetch_sgpamap()

In [98]:
#unzip shp file
sgpa_path =SGPA_PATH
sgpa_member=SGPA_MEMBER
zip_path = os.path.join(sgpa_path, sgpa_member)
            
with zipfile.ZipFile(zip_path) as zip:
    zip.extractall(sgpa_path)
    zip.close()
    os.remove(zip_path)

Now that the dataset is ready, lets explore and prepare our data.